In [1]:
import os,sys
import numpy as np
import gdspy as gp
import gdstools as tools
ratio = 1
lane = 300

# When work with mask, make sure you put the cell center at it's box's geometry center, or put it somewhere reasonable and easy to remember. 
# The location of an object is controlled by `wafer.add(gp.CellReference(<cell name>,(X,Y)))`
# Try to keep all geometry in the same layer in one sub-cell. Don't make the whole structure too messy.

outdir = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified'
if not os.path.exists(outdir):
    os.makedirs(outdir)
print('Output dir: %s'%outdir)

Output dir: /Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified


In [2]:
mask = gp.Cell('MASK')

# get kids and reference masks
gdsii=gp.GdsLibrary()

inputmask = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified/nukid_shrink10000.gds' 
gl=gdsii.read_gds(inputmask, rename_template='{name}_sq')
wafer_sq = gl.extract('WAFER_sq')
bb_sq=wafer_sq.get_bounding_box()

inputmask = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified/nukid_shrink10000_cpw.gds' 
gl=gdsii.read_gds(inputmask, rename_template='{name}_sq_cpw')
wafer_sq_cpw = gl.extract('WAFER_sq_cpw')

inputmask = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified/slim_nuKid_2.gds' 
gl=gdsii.read_gds(inputmask, rename_template='{name}_slim')
wafer_slim = gl.extract('WAFER_slim')
bb_slim=wafer_slim.get_bounding_box()

inputmask = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified/slim_nuKid_no_kids.gds' 
gl=gdsii.read_gds(inputmask, rename_template='{name}_slim_cpw')
wafer_slim_cpw = gl.extract('WAFER_slim_cpw')

inputmask = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified/dicealign_ref/HVeV_v3_Mask_mk16.gds'
gl=gdsii.read_gds(inputmask)


/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_79078/1530724647.py:8: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  wafer_sq = gl.extract('WAFER_sq')
/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_79078/1530724647.py:13: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  wafer_sq_cpw = gl.extract('WAFER_sq_cpw')
/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_79078/1530724647.py:17: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  wafer_slim = gl.extract('WAFER_slim')
/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_79078/1530724647.py:22: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  wafer_slim_cpw = gl.extract('WAFER_slim_cpw')


In [3]:
# mask base --> the basic layout of the 4-inch wafer
maskbase = gp.Cell('MASKBASE')
baselayer = 3
lanelayer = 0

# probe units
inputmask_4p = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified/four_point_probe.gds' 
inputmask_vdp = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified/vdp_2.gds' 


## outer frame
parts=['ul', 'ur', 'll', 'lr']

for partname in parts:
    # load the pre-saved points for drawing the polygon 
    maskbase_pts = np.genfromtxt('./polygonpts_maskbase_%sc.txt'%partname)
    # remove the void for devices won't appear on our current mask
    unq, count = np.unique(maskbase_pts, axis=0, return_counts=True)
    repeated_groups=unq[count>1]
    for repeated_group in repeated_groups:
        repeated_idx = np.argwhere(np.all(maskbase_pts == repeated_group, axis=1))
        if repeated_idx.ravel().size==0:
            continue
        else:
            maskbase_pts = np.delete(maskbase_pts, range(repeated_idx.ravel()[0],repeated_idx.ravel()[1]), axis=0)
    # draw the polygon and add it to the mask
    maskbase_pg  = gp.Polygon(maskbase_pts, layer=baselayer)
    
    if partname=='ul':
        w,h = 650.,500.
        gl=gdsii.read_gds(inputmask_4p, rename={'four_point_probe':'FOURPOINTPROBE_NbFd'},layers={2:2, 3:2, 4:2})
        probe4p = gl.extract('FOURPOINTPROBE_NbFd')
        for x in [-32000,-25000]:
            for y in [25000,28500,32000]:
                rect = gp.Rectangle([x+w/2,y+h/2],[x-w/2,y-h/2], layer=baselayer)
                maskbase_pg = gp.boolean(maskbase_pg, rect, "not", layer=baselayer)
                mask.add(gp.CellReference(probe4p,(x,y)))
            
    
    if partname=='ll':
        w,h = 650.,500.
        gl=gdsii.read_gds(inputmask_4p, rename={'four_point_probe':'FOURPOINTPROBE_Al'},layers={2:1, 3:2, 4:2})
        probe4p = gl.extract('FOURPOINTPROBE_Al')
        for x in [-32000,-25000]:
            for y in [-25000,-28500,-32000]:
                rect = gp.Rectangle([x+w/2,y+h/2],[x-w/2,y-h/2], layer=baselayer)
                maskbase_pg = gp.boolean(maskbase_pg, rect, "not", layer=baselayer)
                mask.add(gp.CellReference(probe4p,(x,y)))
                
    if partname=='ur':
        w,h = 650.,500.
        gl=gdsii.read_gds(inputmask_vdp, rename={'vdp':'VDP_Al'},layers={1:1, 3:2, 4:2})
        probevdp = gl.extract('VDP_Al')
        for x in [24000,28000]:
            for y in [34000,38000]:
                rect = gp.Rectangle([x+w/2,y+h/2],[x-w/2,y-h/2], layer=baselayer)
                maskbase_pg = gp.boolean(maskbase_pg, rect, "not", layer=baselayer)
                mask.add(gp.CellReference(probevdp,(x,y)))
        for x in [34000,38000]:
            for y in [24000,28000]:
                rect = gp.Rectangle([x+w/2,y+h/2],[x-w/2,y-h/2], layer=baselayer)
                maskbase_pg = gp.boolean(maskbase_pg, rect, "not", layer=baselayer)
                mask.add(gp.CellReference(probevdp,(x,y)))
    
    if partname=='lr':
        w,h = 650.,500.
        gl=gdsii.read_gds(inputmask_vdp, rename={'vdp':'VDP_NbFd'},layers={1:2, 3:2, 4:2})
        probevdp = gl.extract('VDP_NbFd')
        for x in [24000,28000]:
            for y in [-34000,-38000]:
                rect = gp.Rectangle([x+w/2,y+h/2],[x-w/2,y-h/2], layer=baselayer)
                maskbase_pg = gp.boolean(maskbase_pg, rect, "not", layer=baselayer)
                mask.add(gp.CellReference(probevdp,(x,y)))
        for x in [34000,38000]:
            for y in [-24000,-28000]:
                rect = gp.Rectangle([x+w/2,y+h/2],[x-w/2,y-h/2], layer=baselayer)
                maskbase_pg = gp.boolean(maskbase_pg, rect, "not", layer=baselayer)
                mask.add(gp.CellReference(probevdp,(x,y)))
        
    maskbase.add(maskbase_pg)
    
## dicing lanes
dx=abs(bb_sq[0][0]-bb_sq[1][0]) + lane
dy=abs(bb_sq[0][1]-bb_sq[1][1]) + lane

for ix in range(1,5):
    x = ix*dx
    for iy in range(-4,5):
        y = iy*dy
        
        if (abs(x)+abs(y)>61800) or (abs(x-dx)+abs(y)>61800) or (abs(x)>50000) or (abs(x-dx)>50000):
            maskbase_pp = None
        else:
            maskbase_pp = gp.PolyPath([(x-dx,y), (x,y)], 30, layer=baselayer)
            maskbase.add(maskbase_pp)
            maskbase_pplane = gp.PolyPath([(x-dx,y), (x,y)], lane, layer=lanelayer)
            maskbase.add(maskbase_pplane)
            
        if (abs(x)+abs(y)>61800) or (abs(x)+abs(y-dy)>61800) or (abs(y)>50000) or (abs(y-dy)>50000):
            maskbase_pp = None
        else:
            maskbase_pp = gp.PolyPath([(x,y-dy), (x,y)], 30, layer=baselayer)
            maskbase.add(maskbase_pp)
            maskbase_pplane = gp.PolyPath([(x,y-dy), (x,y)], lane, layer=lanelayer)
            maskbase.add(maskbase_pplane)
        
dx=abs(bb_slim[0][0]-bb_slim[1][0]) + lane
dy=abs(bb_slim[0][1]-bb_slim[1][1]) + lane

for ix in [0,-1,-2]:
    x = ix*dx
    for iy in range(-12,13):
        y = iy*dy
        
        if (abs(x)>19800 and abs(y)>21300) or (abs(x)>39600) or (abs(y)>42600) or (abs(x-dx)>19800 and abs(y)>21300) or (abs(x-dx)>39600):
            maskbase_pp = None
        else:
            maskbase_pp = gp.PolyPath([(x-dx,y), (x,y)], 30, layer=baselayer)
            maskbase.add(maskbase_pp)
            maskbase_pplane = gp.PolyPath([(x-dx,y), (x,y)], lane, layer=lanelayer)
            maskbase.add(maskbase_pplane)
        
        if (abs(x)>19800 and abs(y)>21300) or (abs(x)>39600) or (abs(y)>42600) or (abs(x)>19800 and abs(y-dy)>21300) or (abs(y-dy)>42600):
            maskbase_pp = None
        else:
            maskbase_pp = gp.PolyPath([(x,y-dy), (x,y)], 30, layer=baselayer)
            maskbase.add(maskbase_pp)
            maskbase_pplane = gp.PolyPath([(x,y-dy), (x,y)], lane, layer=lanelayer)
            maskbase.add(maskbase_pplane)
        

/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_79078/4269366281.py:31: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  probe4p = gl.extract('FOURPOINTPROBE_NbFd')
/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_79078/4269366281.py:52: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  probevdp = gl.extract('VDP_Al')
/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_79078/4269366281.py:42: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  probe4p = gl.extract('FOURPOINTPROBE_Al')
/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_79078/4269366281.py:67: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  probevdp = gl.extract('VDP_NbFd')
/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_79078/4269366281.py:93: DeprecationWarning: [GDSPY] PolyPath is deprecated favor of FlexPath and will be remo

In [4]:
# alignment marks
alignmarks=gl.extract('Alignment_Marks').flatten()
alignmarks_new=gp.Cell('ALIGNMARKS')

pgs = alignmarks.polygons
for pg in pgs:
    if pg.layers[0]==1:
        p1=gp.Polygon(pg.polygons[0],layer=1)
        alignmarks_new.add(p1)
        pg.rotate(90*np.pi/180)
        p4=gp.Polygon(pg.polygons[0],layer=4)
        alignmarks_new.add(p4)
    else:
        alignmarks_new.add(gp.Polygon(pg.polygons[0],layer=pg.layers[0]))
        
    
maskbase.add(alignmarks_new)

/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_79078/3803023830.py:2: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  alignmarks=gl.extract('Alignment_Marks').flatten()


In [5]:
# 

In [6]:
# final mask
# mask = gp.Cell('MASK')

mask.add(gp.CellReference(maskbase,(0,0)))

#NK=nukid, 2302=Feb2023, X=undefined date, CL/CQ=chip number slim/square
serlayer=2
ic = 0
dx=abs(bb_sq[0][0]-bb_sq[1][0]) + lane
dy=abs(bb_sq[0][1]-bb_sq[1][1]) + lane

# square
for ix in range(1,5):
    x = (ix-0.5*ix/abs(ix)) * dx
    for iy in range(-4,5):
        if not iy:
            continue
        if abs(ix) + abs(iy)>6:
            continue

        y = (iy-0.5*iy/abs(iy)) * dy
        
        if (abs(ix)==1 and ix+iy==0) or (abs(ix)==3 and abs(iy)==3):
            mask.add(gp.CellReference(wafer_sq_cpw,(x,y),x_reflection=(x>0),rotation=180*(x>0)))
        else:
            mask.add(gp.CellReference(wafer_sq,(x,y),x_reflection=(x>0),rotation=180*(x>0)))
        ic +=1
        text = gp.Text('NK2302XCQ%02d'%ic, 200, ((x-dx/2+1000), (y-dy/2+200)), layer=serlayer)
        mask.add(text)


ic = 0
dx=abs(bb_slim[0][0]-bb_slim[1][0]) + lane
dy=abs(bb_slim[0][1]-bb_slim[1][1]) + lane

for ix in [-1,-2]:
    x = (ix-0.5*ix/abs(ix)) * dx
    
    for iy in range(-12, 13):
        if not iy:
            continue
        if (ix == -2) and (abs(iy) > 6):
            continue
        y = (iy-0.5*iy/abs(iy)) * dy
        if (abs(ix)==1 and ix+iy==0) or (abs(ix)==2 and abs(iy)==6):
            mask.add(gp.CellReference(wafer_slim_cpw,(x,y),x_reflection=(x>0),rotation=180*(x>0)))
        else:
            mask.add(gp.CellReference(wafer_slim,(x,y),x_reflection=(x>0),rotation=180*(x>0)))
        ic +=1
        text = gp.Text('NK2302XCL%02d'%ic, 200, ((x-dx/2+1000), (y-dy/2+200)), layer=serlayer)
        mask.add(text)
        
#gp.write_gds('%s/nukid_NK2302X.gds'%outdir, cells=sorted(tools.GetSubcellNames(mask)))
#gp.write_gds('%s/nukid_NK2302Xb.gds'%outdir, cells=sorted(tools.GetSubcellNames(mask)))
gp.write_gds('%s/nukid_NK2302Xc.gds'%outdir, cells=sorted(tools.GetSubcellNames(mask)))